# **Normalizzazione dei Nomi Aziendali**

## **Introduzione**

Nel processo di deduplicazione e record linkage, è essenziale normalizzare i nomi aziendali per evitare variazioni dovute a abbreviazioni, formati o errori di battitura. In questa fase, implementiamo una strategia per rendere i nomi uniformi e comparabili.

---

## **Step 1: Importazione delle Librerie e del DataFrame**

In [3]:
import re
import pandas as pd
import unidecode

AZIENDE_CSV = '../aziende.csv'
IMPIEGATI_CSV = '../impiegati.csv'
companies_df = pd.read_csv(AZIENDE_CSV)

/tmp/ipykernel_292231/3147525914.py:7: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  companies_df = pd.read_csv(AZIENDE_CSV)


---

## **Step 2: Definizione della Mappa di Sostituzione**
Utilizziamo un dizionario per standardizzare suffissi comuni nei nomi aziendali.
### **Questo va sicuramente migliorato magari passando tutta la prima colonna a chat**

In [4]:
REPLACEMENTS = {
    r"&": "",
    r"(?<=\s|,|\.)inc\.?\b": "",
    r"(?<=\s|,|\.)ltd\b": "",
    r"(?<=\s|,|\.)llc\b": "",
    r"(?<=\s|,|\.)plc\b": "",
    r"(?<=\s|,|\.)co\.?\b": "",  # Handles "co." and "co,"
    r"(?<=\s|,|\.)corp\.?\b": "",
    r"(?<=\s|,|\.)srl\b": "",
    r"(?<=\s|,|\.)spa\b": "",
    r"(?<=\s|,|\.)ag\b": "",
    r"(?<=\s|,|\.)sa\b": "",
    r"(?<=\s|,|\.)ab\b": "",
    r"(?<=\s|,|\.)ou\b": "",
    r"(?<=\s|,|\.)as\b": "",
    r"(?<=\s|,|\.)nv\b": "",
    r"(?<=\s|,|\.)gmbh\b": "",
    r"(?<=\s|,|\.)holding\b": "",
    r"(?<=\s|,|\.)holdings\b": "",
    r"(?<=\s|,|\.)corporation\b": "",
    r"(?<=\s|,|\.)industries\b": "",
    r"(?<=\s|,|\.)company limited\b": "",
    r"(?<=\s|,|\.)technology\b": "",
    r"(?<=\s|,|\.)solutions\b": "",
    r"(?<=\s|,|\.)systems\b": "",
    r"(?<=\s|,|\.)services\b": "",
    r"(?<=\s|,|\.)public company\b": "",
    r"(?<=\s|,|\.)limited\b": "",
    r"(?<=\s|,|\.)group\b": "",
    r"(?<=\s|,|\.)company\b": "",

    # New patterns to remove dotted abbreviations
    r"(?<=\s|,|\.)s\.?p\.?a\.?\b": "",  # Handles "S.p.A." and variations
    r"(?<=\s|,|\.)s\.?r\.?l\.?\b": "",  # Handles "S.R.L."
    r"(?<=\s|,|\.)s\.?a\.?\b": "",      # Handles "S.A."
    r"(?<=\s|,|\.)n\.?v\.?\b": ""       # Handles "N.V."
}

---

## **Step 3: Funzione di Normalizzazione**

Questa funzione applica una serie di trasformazioni per ottenere nomi uniformi:

- Conversione in minuscolo
- Rimozione di accenti
- Eliminazione di caratteri speciali
- Sostituzione dei suffissi tramite `REPLACEMENTS`

In [5]:
def normalize_name(name):
    """Normalizza i nomi delle aziende."""
    if pd.isna(name) or not isinstance(name, str):
        return ""
    
    # 1. Rimuove accenti e caratteri speciali
    name = unidecode.unidecode(name)
    
    # 2. Minuscolo e rimozione spazi extra
    name = name.lower().strip()
    
    # 3. Rimuove testo tra virgolette
    name = re.sub(r'"([^"]+)"','', name) # 3. Rimuove testo tra virgolette
    
    # 4. Applica le sostituzioni dalla mappa REPLACEMENTS
    for pattern, replacement in REPLACEMENTS.items():
        name = re.sub(pattern, replacement, name)
    
    # 5. Pulisce caratteri inutili
    name = re.sub(r"\s*,+\s*", " ", name)
    name = re.sub(r"(?<!\w)\.+(?!\w)", " ", name)
    
    # 6. Sostituisce spazi multipli con uno solo
    name = re.sub(r'\s+', ' ', name)  
    
    # 7. Eliminare punteggiatura indesiderata alla fine 
    name = re.sub(r"[.,\-]+$", "", name)
    
    return name.strip()

---

## **Step 4: Applicazione alla Colonna 'company_name'**

Eseguiamo la normalizzazione sulla colonna contenente i nomi aziendali.

In [6]:
companies_df['normalized_name'] = companies_df['company_name'].apply(normalize_name)
companies_df['modified'] = companies_df['normalized_name'] != companies_df['company_name']

---

## **Step 5: Visualizzazione e salvataggio del Risultato**

Mostriamo i primi risultati per verificare la trasformazione
e salva tutto in un csv

In [7]:
num_modified = companies_df['modified'].sum()
total_entries = len(companies_df)
percentage_modified = (num_modified / total_entries) * 100

print(f"Entries modificate: {num_modified} su {total_entries} ({percentage_modified:.2f}%)")

modified_entries = companies_df[companies_df['modified']][['company_name', 'normalized_name']]
display(modified_entries.sample(25))

companies_df = companies_df.sort_values(by=['normalized_name'])
companies_df[['company_name', 'normalized_name', 'modified']].to_csv('normalized_names.csv', index=False)

Entries modificate: 44141 su 76808 (57.47%)


,company_name,normalized_name
68991,u gutka ltd,u gutka
20118,ecograf limited,ecograf
69714,"universal health services, inc.",universal health
68280,"trump corporation, the",trump the
34553,japan real estate investment corp,japan real estate investment
37449,"kuo yang construction co.,ltd",kuo yang construction
3727,amber group,amber
48626,p. a. k. o. hui ltd,p. a. k. o. hui
68144,trinity watthana public company ltd,trinity watthana
71550,vibropower corp ltd,vibropower


---

# **Normalizzazione dei paesi**

In [8]:
filtered_df = companies_df[companies_df['country'].notna() & (companies_df['country'].str.strip() != "")].copy()
display(filtered_df[['company_name', 'country']].sample(25))

,company_name,country
65646,teleport,united states
49841,pharma mar,spain
2182,ajinomoto,japan
67132,tmt,italy
18272,derwent london,uk
12047,candy hoover group srl,it00786860965
29417,harmonic,usa
17303,cyrela brazil realty empreendimentos e partici...,brazil
39562,lupin,india
56430,safestore,uk


In [9]:
iva_country_df = filtered_df[filtered_df['country'].astype(str).str.contains(r"\d", na=False)].copy()
display(iva_country_df[['company_name', 'country']])

,company_name,country
79,2i rete gas spa,it06724610966
85,2v energy srl,it03795470248
294,a. menarini industrie farmaceutiche riunite srl,it00395270481
303,a.i.a. agricola italiana alimentare spa,it00233470236
319,a.r.g.o. spa,it01327400352
...,...,...
75075,zanetti spa,it00373690163
75193,zeor finanziaria spa,it05678591008
75502,zignago holding spa,it03781170281
75659,zucchetti group spa,it04171890157


## **Step 1: Eliminazione delle partite IVA dai paesi**

Le partite iva sono codici da 11 cifre e possono essere precedute da due lettere iniziali rappresentanti il paese (e.g. "it") 

In [10]:
def remove_piva_from_country(country):
    if pd.isna(country) or not isinstance(country, str):
        return country
    vat_regex = r"(?:IT|it)?\d{11}"
    if re.fullmatch(vat_regex, country):
        return "italy"
    return country

---

## **Step 2: Traduzione dei nomi dei paesi in inglese**

Eliminazione di caratteri particolari come parentesi e virgolette dal campo country

In [11]:
def clean_country(country): 
    if pd.isna(country) or not isinstance(country, str):
        return ""
    
    # Rimuove parentesi tonde ( ), quadre [ ] e singole virgolette '
    country = re.sub(r"[\[\]']", "", country)  
    country = re.sub(r"\s*\(.*?\)\s*", "", country).strip()  
        
    return country

In [12]:
import pycountry

country_mapping = {
    'it': 'italy',            # Codici e abbreviazioni per l'Italia
    'uk': 'united kingdom',   # Abbreviazione per il Regno Unito
    'usa': 'united states',   # Abbreviazione per gli Stati Uniti
    'us': 'united states',    # Abbreviazione per gli Stati Uniti (altro formato)
    'fr': 'france',           # Abbreviazione per la Francia
    'de': 'germany',          # Abbreviazione per la Germania
    'es': 'spain',            # Abbreviazione per la Spagna
    'ca': 'canada',           # Abbreviazione per il Canada
    'au': 'australia',        # Abbreviazione per l'Australia
    'cn': 'china',            # Abbreviazione per la Cina
    'in': 'india',            # Abbreviazione per l'India
    'jp': 'japan',            # Abbreviazione per il Giappone
    'br': 'brazil',           # Abbreviazione per il Brasile
    'kr': 'south korea',      # Abbreviazione per la Corea del Sud
    'mx': 'mexico',           # Abbreviazione per il Messico
    'se': 'sweden',           # Abbreviazione per la Svezia
    'no': 'norway',           # Abbreviazione per la Norvegia
    'fi': 'finland',          # Abbreviazione per la Finlandia
    'nl': 'netherlands',      # Abbreviazione per i Paesi Bassi
    'ch': 'switzerland',      # Abbreviazione per la Svizzera
    'ru': 'russia',           # Abbreviazione per la Russia
    'sa': 'south africa',     # Abbreviazione per il Sudafrica
    'kr': 'south korea',      # Abbreviazione per la Corea del Sud
    'sg': 'singapore',        # Abbreviazione per Singapore
    "uae": "united Arab Emirates",
}

In [13]:
def translate_country(country):
    if pd.isna(country) or not isinstance(country, str) or country.strip() == "":
        return ""
    
    if country in country_mapping:
        return country_mapping[country]
    
    try:
        country_obj = pycountry.countries.lookup(country)
        return country_obj.name
    except LookupError:
        return country

In [14]:
def normalize_country(country):
    if pd.isna(country) or not isinstance(country, str) or country.strip() == "":
        return ""
    
    # Se ci sono più paesi separati da virgole, li normalizziamo singolarmente
    country_list = [c.lower().strip() for c in country.split(",")]

    normalized_countries = []
    for c in country_list:
        c = clean_country(c)
        c = remove_piva_from_country(c)
        c = translate_country(c)
        normalized_countries.append(c.lower().strip())
    
    return ", ".join(normalized_countries)

In [15]:
companies_df['normalized_country'] = companies_df['country'].apply(normalize_country)
companies_df['modified'] = companies_df['normalized_country'] != companies_df['country']

In [16]:
# Exclude NaN to "" modifications
valid_modifications = companies_df[
    (companies_df["modified"]) & ~(companies_df["country"].isna() & (companies_df["normalized_country"] == ""))
]

num_modified = len(valid_modifications)
total_entries = len(companies_df)
percentage_modified = (num_modified / total_entries) * 100

print(f"Entries modificate: {num_modified} su {total_entries} ({percentage_modified:.2f}%)")

modified_entries = valid_modifications[['country', 'normalized_country']]
display(modified_entries.sample(25))

companies_df = companies_df.sort_values(by=['normalized_country'])
companies_df[['country', 'normalized_country', 'modified']].to_csv('normalized_country.csv', index=False)

Entries modificate: 8258 su 76808 (10.75%)


,country,normalized_country
66974,usa,united states
32378,it02896940273,italy
69510,usa,united states
50928,usa,united states
37888,usa,united states
49295,usa,united states
11673,usa,united states
10879,usa,united states
31497,usa,united states
60267,usa,united states


# **Normalizzazione delle città**

In [17]:
filtered_df = companies_df[companies_df['city'].notna() & (companies_df['city'].str.strip() != "")].copy()
display(filtered_df[['company_name', 'city']].sample(25))

,company_name,city
13172,cfc underwriting limited,london
8520,beam global international limited,glasgow
30320,hilton international hotels (uk) limited,watford
30607,"hobsons, inc.",cincinnati
54450,redesign health,new york
69083,uala,buenos aires
39339,louvre hotels italia srl,rivoli
2166,"aj perri holdco, llc",tinton falls
60150,sky limited,isleworth
66194,the brand group limited,london


### **Step 1: Eliminare eventuale testo tra parentesi** 
La cosa che sempra più intuitiva da fare è eliminare eventuale testo tra parentesi dopo la città (perdendo un po' di informazione, ma comunque accettabile)

In [18]:
def clean_city(city):
    if pd.isna(city) or not isinstance(city, str):
        return ""
    
    city = re.sub(r"\s*\(.*?\)\s*", "", city).strip()
    return city 

### **Step 2: Tradurre nomi di città italiane in italiano** 

Bisognerebbe anche indagare se qualche città viene riportata in lingue diverse...

In [19]:
# Query in italiano
milano_df = filtered_df[filtered_df["city"] == "milano"]
roma_df = filtered_df[filtered_df["city"] == "roma"]

# Query in inglese
milan_df = filtered_df[filtered_df["city"] == "milan"]
rome_df = filtered_df[filtered_df["city"] == "rome"]

In [20]:
display(milano_df[['company_name', 'city']])
display(milan_df[['company_name', 'city']])

,company_name,city
4619,apeiron societa' benefit spa,milano
9202,belenergia spa,milano
9203,belenergia spa,milano
9200,belenergia mezz finance srl,milano
9201,belenergia mezz finance srl,milano
...,...,...
11664,c.w.f. children worldwide fashion italia srl,milano
15446,colas rail italia spa,milano
55608,rothschild & c. italia spa,milano
19537,dual italia spa,milano


,company_name,city
57140,satispay,milan
57364,scalapay,milan


In [21]:
display(roma_df[['company_name', 'city']])
display(rome_df[['company_name', 'city']])

,company_name,city
2463,ald automotive italia srl,roma
4282,angelini pharma italia aziende chimiche riunit...,roma
9423,bester new energy italia srl,roma
10529,bombardier transportation italy spa,roma
69748,universita' degli studi roma tre,roma
...,...,...
17626,daniela y srl,roma
43417,mts spa,roma
26231,gestore dei servizi energetici - gse spa,roma
26226,gestione concessioni spa,roma


,company_name,city


Probabilmente, visto la sorgente dati, le aziende italiane sono state estratte da repository italiani o almeno convertite con città in italiano. Solo due aziende (*satispay* e *scalapay*) hanno il nome riportato in inglese.

Le città italiane sono segnate maggiormente in italiano possiamo pensare di **convertire le città con nome inglese in italiano** 

Abbiamo bisogno di un dizionario che mappi i nomi delle città (City-mapping). Evitiamo di scriverlo da zero e ne usiamo uno noto da qualche libreria (*do not reinvent the wheel...*).

Usiamo dunque la libreria `geopy` che con **GeoNames**, un database globale di località, ci permette di convertire i nomi 

In [22]:
from geopy.geocoders import Nominatim
import time
from tqdm import tqdm
import pickle

CITY_CACHE_FILE = "geopy_city_cache.pkl"
tqdm.pandas()
geolocator = Nominatim(user_agent="city_normalizer")

try:
    with open(CITY_CACHE_FILE, 'rb') as f:
        city_cache = pickle.load(f)
    print("Found saved cache...")
except FileNotFoundError:
    city_cache = {}
    print("Cache file not found, creating a new one...")

cache_hit = 0
cache_miss = 0

Found saved cache...


In [23]:
def normalize_city(city):
    global cache_hit, cache_miss
    city = clean_city(city)

    if city in city_cache:
        cache_hit += 1
        return city_cache[city]
    
    cache_miss += 1
    
    try:
        time.sleep(0.5)
        location = geolocator.geocode(city, exactly_one=True, timeout=10)
        if location and "Italia" in location.address:
            location_it = geolocator.geocode(city, language="it", timeout=10)
            if location_it:
                it_city = location_it.raw.get("display_name", city).split(",")[0].lower().strip()
                city_cache[city] = it_city
                return it_city
    except Exception as e:
        print(f"Errore con {city}: {e}")

    city_cache[city] = city
    return city

In [24]:
def tqdm_wrapper(row, tqdm_bar):
    if pd.isna(row['city']) or not isinstance(row['city'], str) or row['city'] == "":
        tqdm_bar.update(1)
        return ""    
    
    global cache_miss, cache_hit
    normalized = normalize_city(row['city']) if row['country'] == "italy" else row['city']
    total_req = cache_hit + cache_miss
    hit_ratio = (cache_hit / total_req) * 100 if total_req > 0 else 0
    tqdm_bar.set_description(f"Processing | Cache Hit Ratio: {hit_ratio:.2f}%")  # Update bar without new lines
    tqdm_bar.update(1)

    return normalized

**GeoNames** funziona, adesso applichiamolo all'intero dataframe

In [25]:
with tqdm(total=len(companies_df), desc="Processing", unit="entry") as tqdm_bar:
        companies_df['city_translate'] = companies_df.apply(lambda row: tqdm_wrapper(row, tqdm_bar), axis=1)

with open(CITY_CACHE_FILE, 'wb') as f:
        pickle.dump(city_cache, f)
        
companies_df['modified'] = companies_df['city'] != companies_df['city_translate']

Processing | Cache Hit Ratio: 100.00%: 100%|██████████| 76808/76808 [00:08<00:00, 8884.62entry/s] 


In [26]:
valid_modifications = companies_df[
    (companies_df["modified"]) & ~(companies_df["city"].isna() & (companies_df["city_translate"] == ""))
]

num_modified = len(valid_modifications)
total_cities = len(companies_df)
percentage_modified = (num_modified / total_cities) * 100

print(f"Città tradotte: {num_modified} su {total_cities} ({percentage_modified:.2f}%)")

translated_cities = valid_modifications[['normalized_name', 'city', 'city_translate']]
display(translated_cities.sample(25))

companies_df = companies_df.sort_values(by=['city'])
companies_df[['normalized_name', 'city', 'city_translate']].to_csv('translated_cities.csv', index=False)

Città tradotte: 170 su 76808 (0.22%)


,normalized_name,city,city_translate
12486,casa damiani,valenza (ex viale santuario 46 (variazione top...,valenza
27795,green power,"sassocorvaro auditore (caprazzino, (ex sassoco...",sassocorvaro auditore
40592,marposs,bentivoglio (san marino ),bentivoglio
70834,valentino,valdagno (maglio ),valdagno
72245,vossloh-schwabe italia,sarsina (piano ),sarsina
73457,witor's,corte de' frati (aspice ),corte de' frati
23321,filmmaster productions,roma (st. dott. amerigo casamonti ),roma
22963,fca italy in forma fiat chrysler automobiles e...,torino (quartiere mirafiori ),torino
73168,whirlpool emea,fabriano (ca' maiano ),fabriano
4498,antica farmaceutica lucchese,lucca (san vito ),lucca


---

### **Salvataggio delle modifiche sul csv**

In [27]:
companies_df['country'] = companies_df['normalized_country']
companies_df['company_name'] = companies_df['normalized_name']
companies_df['city'] = companies_df['city_translate']

companies_df.drop(columns=['normalized_name', 'normalized_country', 'city_translate', 'modified'])
companies_df = companies_df.sort_values(by=['company_name'])
companies_df.drop_duplicates(inplace=True)                          # drop duplicates
companies_df.to_csv('../aziende_normalizzate.csv', index=False)